# base_line 모델(roberta-base)의 선정 및 f1_score 

Team LostCow의 roberta모델을 기준으로 함

출처: https://github.com/l-yohai/KLUE

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 7.0 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 6.5 MB 46.4 MB/s 
     |████████████████████████████████| 895 kB 10.9 MB/s 
     |████████████████████████████████| 596 kB 3.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 8.5 MB/s 
     |████████████████████████████████| 212 kB 71.0 MB/s 
     |████████████████████████████████| 134 kB 76.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.9 MB/s 
     |████████████████████████████████| 127 kB 73.6 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 75.1 MB/s 
     |████████████████████████████████| 144 kB 73.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3

In [ ]:
import tarfile

#훈련 데이터셋 다운로드 및 압축풀기

!wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

tar_bz2_file = tarfile.open("/content/klue-sts-v1.1.tar.gz")
tar_bz2_file.extractall(path="/content")
tar_bz2_file.close()

--2022-03-17 00:19:46--  https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz
Resolving aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)... 52.218.233.203
Connecting to aistages-prod-server-public.s3.amazonaws.com (aistages-prod-server-public.s3.amazonaws.com)|52.218.233.203|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1349881 (1.3M) [application/x-gzip]
Saving to: ‘klue-sts-v1.1.tar.gz’

klue-sts-v1.1.tar.g 100%[===================>]   1.29M  --.-KB/s    in 0.09s   

2022-03-17 00:19:46 (14.4 MB/s) - ‘klue-sts-v1.1.tar.gz’ saved [1349881/1349881]



In [ ]:
import torch

#GPU 사용 여부 확인 및 name 확인

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [ ]:
import json

#train 데이터셋을 train,valid 데이터셋으로 split

def read_json(file_path):
    with open(file_path) as f:
        return json.load(f)

data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_train.json')

train_length=int(len(data)*0.9)
train=data[:train_length]
vaild=data[train_length:]

print('data_set:',len(data))
print('train_set:',len(train),', valid_set:',len(vaild))

with open('train_split.json','w') as f:
  json.dump(train,f,ensure_ascii = False)
with open('valid_split.json','w') as f:
  json.dump(vaild,f,ensure_ascii = False)

data_set: 11668
train_set: 10501 , valid_set: 1167


In [ ]:
!git clone https://github.com/l-yohai/KLUE #base line 모델 불러오기

Cloning into 'KLUE'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 313 (delta 52), reused 101 (delta 29), pack-reused 179
Receiving objects: 100% (313/313), 5.79 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [ ]:
# base line 모델 학습
!python /content/KLUE/sts/train.py --data_dir /content --model_name_or_path "klue/roberta-base" --train_filename "train_split.json" --valid_filename "valid_split.json" --num_train_epochs 10 --save_steps 100 --save_steps 100

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForStsRegression: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForStsRegression from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForStsRegression from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForStsRegression were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['cls_fc_layer.linear.weight', 'sentence_fc_layer.linear.bias', 'dense.linear.weight', 'roberta.pool

In [ ]:
mv /content/klue-sts-v1.1.tar.gz /content/model/klue-sts-v1.1.tar.gz

In [ ]:
#학습한 base line 모델을 이용하여 dev_set 예측값 출력(real_label)
!python /content/KLUE/sts/inference.py --test_filename "/content/klue-sts-v1.1/klue-sts-v1.1_dev.json" --output_dir "/content"  --model_tar_file "klue-sts-v1.1.tar.gz"

In [ ]:
#valid의 label 값을 추출

valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['binary-label'] for data in valid_data]

In [ ]:
from sklearn.metrics import classification_report,f1_score
import pandas as pd

#Regression pred 를 binary-label pred로 변환 및 혼돈 메트릭스 및 f1_score 시각화

df_pred = pd.read_csv('/content/output.csv',header=None)

pred=(df_pred>2.5).astype(int)
print("epochs 10 / >2.5")
print(classification_report(valid_label, df_pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, df_pred),end='\n\n')

pred=(df_pred>3.1).astype(int)
print("epochs 10 / >3.1")
print(classification_report(valid_label, df_pred))
print('base_line(roberta-base) f1_score:',f1_score(valid_label, df_pred))

epochs 10 / >2.5
              precision    recall  f1-score   support

           0       1.00      0.66      0.79       299
           1       0.68      1.00      0.81       220

    accuracy                           0.80       519
   macro avg       0.84      0.83      0.80       519
weighted avg       0.86      0.80      0.80       519

base_line(roberta-base) f1_score: 0.8103130755064457

epochs 10 / >3.1
              precision    recall  f1-score   support

           0       0.96      0.80      0.87       299
           1       0.78      0.96      0.86       220

    accuracy                           0.87       519
   macro avg       0.87      0.88      0.87       519
weighted avg       0.89      0.87      0.87       519

base_line(roberta-base) f1_score: 0.859470468431772


In [ ]:
from datasets import load_metric

#dev_set에 대한 pearsonr score 출력 

df_pred = pd.read_csv('/content/output.csv',header=None)
valid_data=read_json('/content/klue-sts-v1.1/klue-sts-v1.1_dev.json')
valid_label=[data['labels']['real-label'] for data in valid_data]

pearson = load_metric("pearsonr").compute
metric = pearson(predictions=df_pred.to_numpy(), references=valid_label)

print(metric)

{'pearsonr': 0.8939478658048167}
